In [15]:
import rosbag
import pandas as pd
bagname  = 'robot_O2_T4_A0_2024-01-09-12-37-55'
bag = rosbag.Bag(f'/home/bioinlab/Documents/real-OMX-bioin-tacto-grasp-and-lift/bags/Object2/{bagname}.bag')
# import rosbag
import pandas as pd
import numpy as np
# Open the ROS bag file
# bag_path = 'your_rosbag_file.bag'
# bag = rosbag.Bag(bag_path)

tf_topics = ['/raw_barometers_teensy1', '/raw_barometers_teensy2', #left teesny 2
             '/robot_instruction']

# Create an empty DataFrame to store the data
columns = ['timestamp', 'baro1', 'baro2', 'instruction']

df = pd.DataFrame(columns=columns)
tf_data = []

# Iterate through the bag and extract tf data

for topic, msg, t in bag.read_messages(topics=tf_topics):
    data = {
        'timestamp': t.to_sec(),
        'baro1': np.nan,
        'baro2': np.nan,
        'instruction': np.nan,

    }

    if topic == '/raw_barometers_teensy1':
        data['baro1'] = msg.baro_level
    elif topic == '/raw_barometers_teensy2':
        data['baro2'] = msg.baro_level
    elif topic == '/robot_instruction':
        data['instruction'] = msg.data

    tf_data.append(data)




sensor_df = pd.DataFrame(tf_data)
sensor_df.set_index('timestamp', inplace=True)

tf = pd.read_csv(f'/home/bioinlab/Documents/real-OMX-bioin-tacto-grasp-and-lift/bags/Object2/tf/{bagname}.csv')
tf =tf.rename(columns={"time": "timestamp", 

                   })
tf.set_index('timestamp', inplace=True)
# join tf and sensor data
result_df = pd.concat([sensor_df, tf], axis=0, sort=True, join='outer')
result_df = result_df.sort_index()

result_df_without_na =result_df.ffill()
index_of_instruction = result_df[result_df['instruction'].notna()]
# taking the sensor info of the next instruction so the instruction alligns with the sensor info returned
result_df_without_na = result_df_without_na[result_df['instruction'].notna()]
result_df_without_na['instruction'] = result_df_without_na['instruction'].shift(-1)
result_df_without_na
result_df_without_na.to_csv(f'/home/bioinlab/Documents/real-OMX-bioin-tacto-grasp-and-lift/bags/Object2/tf/{bagname}_withtf.csv', index=True)
